In [2]:
"The general purpose of this notebook is to be a data wrangling and a ready-on \
Canastra and execute genetic selection of the best individuals"

'The general purpose of this notebook is to be a data wrangling and a ready-on Canastra and execute genetic selection of the best individuals'

In [3]:
#https://stackoverflow.com/questions/44929118/minimizing-multi-objective-function-using-deap

In [46]:
!ls

 animation_frames
 animation.html
 animation.ipynb
 canastra_completo.ipynb
 canastra.ipynb
 canastra_map.png
 canastra_network.pdf
 cpf.csv
 dea-canastra.ipynb
 deap-canastra.ipynb
 dea.py
 df_canastra.csv
 df_canastra_ph3.csv
 df_selected.csv
'Five sector sustainability Model a proposal for assessing sustainability of production systems(1).pdf'
 flow_graph.png
 genetic_graph.png
 multi_criteria_well.ods
 plot.py
 procedencia_canastra.pdf
 __pycache__
 pyDEA
 Questionario.pdf
'Questionário - Queijo Canastra VI - Final - Copia.pdf'
'Spreadsheet Mercosur.xlsm'
'The Impact of Geographical Indication on the Revitalisation of a Regional Economy: A Case.pdf'
 tools.py


In [45]:
from numpy import array
import seaborn as sns
import numpy as np
import random
from deap import base, creator, tools, algorithms
import pandas  as pd
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

#DATA ANALYSIS ENVELOPMENT PACKAGE

import dea as dea
import tools
import plot

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

###  Multi-objective Optimization Problem  ###

raw_df = pd.read_csv("df_canastra.csv").sample(frac=1)

raw_df.head(1)

,QUEST,S_01_GPS_PONTO,S_02_ESTE,S_03_NORTE,S_04_ALTITUDE,S_05_FOTO,S_06_MUNICIPIO,S_07_DISTANCIA_AO_CENTRO,S_08_DISTRITO,S_09_COMUNIDADE,S_10_NOME_ENTREVISTADO,S_11_SEXO,S_12_FUNCAO_PROPRIEDADE,S_13_NOME_PROPRIETARIO,S_14_APELIDO,S_15_IDADE,S_16_SEXO,S_17_TELEFONE_CONTATO,S_18_RG,S_19_CPF,S_20_NOME_PROPRIEDADE,S_21_AREA_PROPRIEDADE,S_22_POSSUI_ENERGIA_ELETRICA,S_23_CASO_POSITIVO_QUANTOS_KVA,S_24_POSSUI_NUMERO_INSCRICAO_PRODUTOR,S_24_1_NUMERO_INSCRICAO,S_25_E_COOPERADO_SARONCRED,S_26_CASO_NAO_COOPERADO_MOTIVO,S_27_E_COOPERADO_A_APROCAN,S_28_CASO_NAO_ASSOCIADO_MOTIVO,S_29_SITUACAO_CADASTRAL_IMA,S_30_NUMERO_CADASTRO_IMA,S_31_CASO_CADASTRADO_IMA_VANTAGENS,S_32_SITUAÇÃO_CADASTRAL_JUNTO_A_SIF,S_33_NUMERO_SIF,S_34_SE_CADASTRADO_SIF_VANTAGENS,S_35_PRINCIPAIS_RACAS_UTILIZADAS,S_36_1_PASTAGEM_BRAQUIÁRIA_%,S_36_2_PASTAGEM_CAPIM_GORDURA_%,S_36_3_PASTAGEM_NATIVA_%,S_36_4_PASTAGEM_PROVISÓRIA_%,S_36_5_PASTAGEM_OUTROS_%,S_37_SISTEMA_DE_PRODUÇÃO,S_38_CASO_SEJA_A_PASTO,S_39_SUPLEMENTAÇÃO_ALIMENTAR,S_40_SUPLEMENTAÇÃO_MINEIRAL,S_41_POSSUI_TRONCO_E_BEZERRO,S_42_POSSUI_SALA_DE_ORDENHA,S_43_SALA_DE_ORDENHA_É_COBERTA,S_44_SALA_DE_ORDENHA_É_CIMENTADA,S_45_POSSUI_ORDENHA_MECÂNICA,S_46_QUANTAS_ORDENHAS_DIÁRIAS,S_47_ORDENHA,S_48_PRODUÇÃO_LEITE_MANHA_DIA,S_49_PRODUÇÃO_LEITE_A_TARDE_DIA,S_50_PRODUÇÃO_TOTAL_DE_LEITE_DIA,S_51_POSSUI_SALA_ÁREA_DE_ESPERA,S_52_CURRAL_É_CALÇADO,S_53_TEM_PONTO_DE_ÁGUA,S_54_SE_POSSUI_A_ÁGUA_É_CLORADA,S_55_POSSUI_IMPLANTADA_AS_BPF,S_56_POSSUI_TREINAMENTO_EM_BPF,S_57_FAZ_LIMPEZA_DOS_UTENSÍLIOS,S_58_FAZ_LIMPEZA_DOS_TETOS_ANTES_DA_ORDENHA,S_59_FAZ_SECAGEM_DO_ÚBERE,S_60_REALIZA_TESTE_MASTITE,S_61_CASO_REALIZE_TESTE_MASTITE_QUAL_FAZ,S_62_FAZ_TRATAMENTO_MASTITE,S_63_FAZ_TRATAMENTO_MASTITE_NAS_VACAS_MAGRAS,S_64_EM_CASO_USO_DE_ANTIBIÓTICOS_FICA_SEM_TIRAR_LEITE,S_65_INFRAESTRUTURA_E_BENFEITORIAS_EXISTENTES,S_66_NÚMERO_VACAS_EM_LACTAÇÃO,S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO,S_68_QUAIS_AS_VACINAS_APLICADAS_NO_REBANHO,S_69_QUAIS_EXAMES_REALIZADOS,S_70_DESTINO_ANIMAIS_SORO_POSITIVOS,S_71_FAZ_MARCAÇÃO_DOS_ANIMAIS,S_72_1_DESTINO_PRODUÇÃO_QUEIJO_LOCAL_%,S_72_2_DESTINO_PRODUÇÃO_QUEIJO_REGIONAL_%,S_72_3_DESTINO_PRODUÇÃO_QUEIJO_ENTREPOSTO_%,S_72_4_DESTINO_PRODUÇÃO_QUEIJO_BH_%,S_72_5_DESTINO_PRODUÇÃO_QUEIJO_SP_CAPITAL_%,S_72_6_DESTINO_PRODUÇÃO_QUEIJO_SP_INTERIOR%,S_72_7_DESTINO_PRODUÇÃO_QUEIJO_RJ%,S_72_6_DESTINO_PRODUÇÃO_QUEIJO_OUTROS%,S_73_FINALIDADE_DO_LEITE,S_74_1_PORCENTAGEM_DA_VENDA_DO_LEITE,S_74_2_PORCENTAGEM_NA_FABRICAÇÃO_DO_QUEIJO,S_75_NOME_DO_QUEIJO,S_76_1_TIPO_QUEIJO_CANASTRA_MERENDEIRO,S_77_1_QUANTIDADE_CANASTRA_MERENDEIRO,S_78_1_PESO_TOTAL_DIA_CANASTRA_MERENDEIRO,S_79_1_PREÇO_MÉDIO_RECEBIDO_CANASTRA_MERENDEIRO,S_76_2_TIPO_QUEIJO_CANASTRA,S_77_2_QUANTIDADE_QUEIJO_CANASTRA,S_78_2_PESO_TOTAL_DIA_QUEIJO_CANASTRA,S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA,S_76_3_TIPO_QUEIJO_CANASTRA_REAL,S_77_3_QUANTIDADE_QUEIJO_CANASTRA_REAL,S_78_3_PESO_TOTAL_DIA_QUEIJO_CANASTRA_REAL,S_79_3_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA_REAL,S_76_4_TIPO_QUEIJO_MEIA_CURA,S_77_4_QUANTIDADE_QUEIJO_MEIA_CURA,S_78_4_PESO_TOTAL_DIA_QUEIJO_MEIA_CURA,S_79_4_PREÇO_MÉDIO_RECEBIDO_QUEIJO_MEIA_CURA,S_76_5_TIPO_QUEIJO_CABACINHO,S_77_5_QUANTIDADE_QUEIJO_CABACINHO,S_78_5_PESO_TOTAL_DIA_QUEIJO_CABACINHO,S_79_5_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CABACINHO,S_76_6_REQUEIJAO,S_77_6_QUANTIDADE_REQUEIJAO,S_78_6_PESO_TOTAL_DIA_REQUEIJAO,S_79_6_PREÇO_MÉDIO_RECEBIDO_REQUEIJAO,S_80_QUANTO_PODERIA_AUMENTAR_SUA_PRODUÇÃO,S_81_QUAL_SERIA_GARGALO_HOJE,S_82_COALHO_UTILIZADO,S_83_MARCA_DO_COALHO,S_84_USA_PINGO,S_85_USA_RALA,S_86_COMO_UTILIZA_O_COALHO,S_87_SALGA,S_88_CASO_SALGUE_ONDE_COLOCA_O_SAL,S_89_1_TEMPO_DE_MATURAÇÃO_ATÉ_03_DIAS_%,S_89_2_TEMPO_DE_MATURAÇÃO_DE_04_A_10_DIAS_%,S_89_3_TEMPO_DE_MATURAÇÃO_DE_11_A_20_DIAS_%,S_89_4_TEMPO_DE_MATURAÇÃO_DE_ACIMA_21_DIAS_%,S_90_TEM_PONTO_DE_ÁGUA_NA_QUEIJARIA,S_91_A_ÁGUA_É_CLORADA,S_92_PRÁTICAS_SANITÁRIAS,S_93_POSSUI_IMPLANTADA_BPF,S_94_POSSUI_TREINAMENTO_EM_BPF,S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO,S_96_COM_QUEM_APRENDEU,S_97_VOCÊ_CONHECE_O_TERMO_INDICAÇÃO_GEOGRÁFICA,S_98

In [5]:
param=[

#Demografico
    
"S_06_MUNICIPIO",
"S_11_SEXO",
"S_13_NOME_PROPRIETARIO",
"S_15_IDADE",
"S_65_INFRAESTRUTURA_E_BENFEITORIAS_EXISTENTES",    
#"S_19_CPF", - depois considerar para a PH3

#Bancario e economico - MBA

"S_24_POSSUI_NUMERO_INSCRICAO_PRODUTOR",
"S_25_E_COOPERADO_SARONCRED",
"S_26_CASO_NAO_COOPERADO_MOTIVO",
"S_27_E_COOPERADO_A_APROCAN",
"S_28_CASO_NAO_ASSOCIADO_MOTIVO",
"S_55_POSSUI_IMPLANTADA_AS_BPF",
"S_134_FAZ_USO_CRÉDITO_RURAL",
"S_135_PARA_QUE_FAZ_USO_CRÉDITO_RURAL",
"S_136_FONTE_DE_RECURSO",
"S_137_MOTIVO_PARA_NÃO_USAR_CRÉDITO_RURAL",
"S_139_PROPRIEDADE_FAZ_CONTROLE_FINANCEIRO",
"S_140_COMO_FAZ_CONTROLE_FINANCEIRO",
"S_127_PODE_AFIRMAR_COM_CERTEZA_LUCRO_POR_UNIDADE_QUEIJO",
"S_141_PROPRIEDADE_FAZ_ALGUM_TIPO_PREVISAO",
"S_142_PROPRIEDADE_POSSUI_COMPUTADOR",
"S_143_A_PROPRIEDADE_POSSUI_INTERNET",
"S_93_POSSUI_IMPLANTADA_BPF",
"S_94_POSSUI_TREINAMENTO_EM_BPF",
"S_96_COM_QUEM_APRENDEU",
"S_131_APENAS_RENDA_QUEIJO_È_SUFICIENTE",


#Sistema K
    
    
"S_21_AREA_PROPRIEDADE",
"S_23_CASO_POSITIVO_QUANTOS_KVA",

"S_50_PRODUÇÃO_TOTAL_DE_LEITE_DIA",
"S_74_2_PORCENTAGEM_NA_FABRICAÇÃO_DO_QUEIJO",
"S_78_2_PESO_TOTAL_DIA_QUEIJO_CANASTRA",
"S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO",
"S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA",
"S_129_QUAL_RENDA_TOTAL_MENSAL_FAMILIAR",
    
    
"S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO",
"S_149_QUANTAS_PESSOAS_CONTRATADAS_TRABALHAM",
"S_148_NUMERO_FAMILIARES_TRABALHAM_PROPRIEDADE"]

In [6]:
select_df = raw_df[param]

## Variable Selection

In [7]:
#object or categorical variable

cat = ["S_06_MUNICIPIO",
"S_11_SEXO",
"S_13_NOME_PROPRIETARIO",
"S_24_POSSUI_NUMERO_INSCRICAO_PRODUTOR",
"S_25_E_COOPERADO_SARONCRED",
"S_26_CASO_NAO_COOPERADO_MOTIVO",
"S_27_E_COOPERADO_A_APROCAN",
"S_28_CASO_NAO_ASSOCIADO_MOTIVO",
"S_55_POSSUI_IMPLANTADA_AS_BPF",
"S_134_FAZ_USO_CRÉDITO_RURAL",
"S_135_PARA_QUE_FAZ_USO_CRÉDITO_RURAL",
"S_136_FONTE_DE_RECURSO",
"S_137_MOTIVO_PARA_NÃO_USAR_CRÉDITO_RURAL",
"S_139_PROPRIEDADE_FAZ_CONTROLE_FINANCEIRO",
"S_140_COMO_FAZ_CONTROLE_FINANCEIRO",
"S_127_PODE_AFIRMAR_COM_CERTEZA_LUCRO_POR_UNIDADE_QUEIJO",
"S_141_PROPRIEDADE_FAZ_ALGUM_TIPO_PREVISAO",
"S_142_PROPRIEDADE_POSSUI_COMPUTADOR",
"S_143_A_PROPRIEDADE_POSSUI_INTERNET",
"S_93_POSSUI_IMPLANTADA_BPF",
"S_94_POSSUI_TREINAMENTO_EM_BPF",
"S_96_COM_QUEM_APRENDEU",
"S_131_APENAS_RENDA_QUEIJO_È_SUFICIENTE"] 

#numerical variable

num = [
"S_65_INFRAESTRUTURA_E_BENFEITORIAS_EXISTENTES",
"S_15_IDADE",
#"S_19_CPF", - depois considerar para a PH3
"S_21_AREA_PROPRIEDADE",
"S_23_CASO_POSITIVO_QUANTOS_KVA",
"S_50_PRODUÇÃO_TOTAL_DE_LEITE_DIA",
"S_74_2_PORCENTAGEM_NA_FABRICAÇÃO_DO_QUEIJO",
"S_78_2_PESO_TOTAL_DIA_QUEIJO_CANASTRA",
"S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO",
"S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA",
"S_129_QUAL_RENDA_TOTAL_MENSAL_FAMILIAR",
"S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO",
"S_149_QUANTAS_PESSOAS_CONTRATADAS_TRABALHAM",
"S_148_NUMERO_FAMILIARES_TRABALHAM_PROPRIEDADE"]

## Outlier Detection and Correction

In [8]:
#A remoção de outliers empregará um intervalo de confiança de 95%

import numpy as np
from scipy import stats

select_df[num] = select_df[num][select_df[num].apply(lambda x: np.abs(x - x.median()) / x.std() < 1.96)]

In [9]:
#Transforming 0 Values to NaN values for Input Strategy

def na_value(x):
    if x == 0:
        return np.nan
    else:
        return x

for i in num:    
    select_df[i] = select_df[i].apply(lambda x : na_value(x))

## Input Strategy for NA values 

In [10]:
#Inputing values for CATEGORICAL Values
for column in cat:
    select_df[column] = select_df[column].fillna(method='ffill').fillna(method='bfill')

In [11]:
#Inputing values for NUMERICAL Values
for column in num:
    select_df[column] = select_df[column].fillna(method='ffill').fillna(method='bfill')

## Data Wrangling to Categorical Values 

In [12]:
#selecting only

for i in cat:
    select_df[i]= select_df[i].str.split(".",expand=True)[0]

## New Variable Creation 

In [13]:
# Soro de leite por dia

for i in tqdm(range(select_df.shape[0])):
    #n=np.random.randint(8,10)
    select_df.loc[i,"WHEY_DAY"] = select_df.loc[i,"S_50_PRODUÇÃO_TOTAL_DE_LEITE_DIA"] * 0.9
    
# CO por dia
    
for i in tqdm(range(select_df.shape[0])):
    n=np.random.randint(75,85)
    select_df.loc[i,"CO_DAY"] = select_df.loc[i,"S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO"]*n    

# WORKFORCE por dia    
    
select_df["TOTAL_WORKFORCE"]=select_df["S_149_QUANTAS_PESSOAS_CONTRATADAS_TRABALHAM"] + select_df["S_148_NUMERO_FAMILIARES_TRABALHAM_PROPRIEDADE"]


#CHEESE PRODUCTION using Proxy as Milk Dailly Litter

for i in tqdm(range(select_df.shape[0])):
    n=np.random.randint(7,9)
    select_df.loc[i,"CHEESE_PROXY"] = (select_df.loc[i,"S_50_PRODUÇÃO_TOTAL_DE_LEITE_DIA"] * select_df.loc[i,"S_74_2_PORCENTAGEM_NA_FABRICAÇÃO_DO_QUEIJO"]/100)/n

# DAILLY REVENUE 

select_df["REVENUE_DAY"]=select_df["S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA"] * select_df["CHEESE_PROXY"]

# DAILLY TAX

dict_tax={0:0.1,
          1:0.15,
          2:0.2,
          3:0.25}

for i in tqdm(range(select_df.shape[0])):
    n=np.random.randint(0,4)
    select_df.loc[i,"TAX_DAY"] = select_df.loc[i,"REVENUE_DAY"]*dict_tax[n]

100%|██████████| 757/757 [00:23<00:00, 36.77it/s]


## Data Normalization (1 Ton of Cheese)

In [14]:
select_df.head(5)

,S_06_MUNICIPIO,S_11_SEXO,S_13_NOME_PROPRIETARIO,S_15_IDADE,S_65_INFRAESTRUTURA_E_BENFEITORIAS_EXISTENTES,S_24_POSSUI_NUMERO_INSCRICAO_PRODUTOR,S_25_E_COOPERADO_SARONCRED,S_26_CASO_NAO_COOPERADO_MOTIVO,S_27_E_COOPERADO_A_APROCAN,S_28_CASO_NAO_ASSOCIADO_MOTIVO,S_55_POSSUI_IMPLANTADA_AS_BPF,S_134_FAZ_USO_CRÉDITO_RURAL,S_135_PARA_QUE_FAZ_USO_CRÉDITO_RURAL,S_136_FONTE_DE_RECURSO,S_137_MOTIVO_PARA_NÃO_USAR_CRÉDITO_RURAL,S_139_PROPRIEDADE_FAZ_CONTROLE_FINANCEIRO,S_140_COMO_FAZ_CONTROLE_FINANCEIRO,S_127_PODE_AFIRMAR_COM_CERTEZA_LUCRO_POR_UNIDADE_QUEIJO,S_141_PROPRIEDADE_FAZ_ALGUM_TIPO_PREVISAO,S_142_PROPRIEDADE_POSSUI_COMPUTADOR,S_143_A_PROPRIEDADE_POSSUI_INTERNET,S_93_POSSUI_IMPLANTADA_BPF,S_94_POSSUI_TREINAMENTO_EM_BPF,S_96_COM_QUEM_APRENDEU,S_131_APENAS_RENDA_QUEIJO_È_SUFICIENTE,S_21_AREA_PROPRIEDADE,S_23_CASO_POSITIVO_QUANTOS_KVA,S_50_PRODUÇÃO_TOTAL_DE_LEITE_DIA,S_74_2_PORCENTAGEM_NA_FABRICAÇÃO_DO_QUEIJO,S_78_2_PESO_TOTAL_DIA_QUEIJO_CANASTRA,S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO,S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA,S_129_QUAL_RENDA_TOTAL_MENSAL_FAMILIAR,S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO,S_149_QUANTAS_PESSOAS_CONTRATADAS_TRABALHAM,S_148_NUMERO_FAMILIARES_TRABALHAM_PROPRIEDADE,WHEY_DAY,CO_DAY,TOTAL_WORKFORCE,CHEESE_PROXY,REVENUE_DAY,TAX_DAY
453,VARGEM BONITA,MASCULINO,ANTONIO DONIZETE DA SILVA,58.0,961.0,SIM,SIM,POUCO CONHECIMENTO,NAO,NAO PERCEBI VANTAGENS,SIM,NÇO,INVESTIMENTO,PRONAF,OUTROS,NAO,ANOTA€OES EM BLOCOS E OU CADERNOS,NAO,NAO,NAO,NAO,NAO,NAO,FAMILIARES,NAO,145.20,1.0,60.0,100.0,12.0,18.0,8.5,2500.0,3.0,2.0,2.0,54.0,1440.0,4.0,7.500000,63.750000,6.375000
117,SÇO ROQUE DE MINAS,MASCULINO,ANTONIO APARECIDA DE LIMA,40.0,393.0,SIM,SIM,POUCO CONHECIMENTO,NAO,NAO PERCEBI VANTAGENS,NAO,NÇO,INVESTIMENTO,PRONAF,MEDO DE NAO ARCAR COM OS COMPROMISSOS,NAO,ANOTA€OES EM BLOCOS E OU CADERNOS,NAO,NAO,NAO,NAO,NAO,NAO,FAMILIARES,SIM,77.84,1.0,90.0,100.0,9.0,18.0,8.5,2000.0,6.0,2.0,2.0,81.0,1494.0,4.0,12.857143,109.285714,21.857143
195,MEDEIROS,MASCULINO,GILBERTO TEIXEIRA DA PAIXAO,52.0,523.0,SIM,SIM,POUCO CONHECIMENTO,NAO,NAO PERCEBI VANTAGENS,NAO,SIM,INVESTIMENTO,PRONAF,MEDO DE NAO ARCAR COM OS COMPROMISSOS,NAO,ANOTA€OES EM BLOCOS E OU CADERNOS,NAO,NAO,NAO,NAO,NAO,NAO,FAMILIARES,SIM,10.00,1.0,120.0,100.0,14.0,22.0,9.0,2500.0,30.0,2.0,1.0,108.0,1826.0,3.0,17.142857,154.285714,15.428571
2,SÇO ROQUE DE MINAS,MASCULINO,JADEAO JOSE PEDRO,60.0,21.0,SIM,NAO,POUCO CONHECIMENTO,NAO,POUCO CONHECIMENTO,NAO,SIM,CUSTEIO,PRONAF,MEDO DE NAO ARCAR COM OS COMPROMISSOS,NAO,ANOTA€OES EM BLOCOS E OU CADERNOS,NAO,NAO,NAO,NAO,NAO,NAO,"AMIGOS, CONHECIDOS",NAO,33.88,1.0,30.0,100.0,3.0,15.0,9.0,1000.0,5.0,2.0,2.0,27.0,1260.0,4.0,3.750000,33.750000,5.062500
589,SÇO ROQUE DE MINAS,MASCULINO,ANTONIO JOSE BARBOSA,77.0,124.0,SIM,SIM,POUCO CONHECIMENTO,NAO,POUCO CONHECIMENTO,NAO,NÇO,CUSTEIO,PRONAF,JUROS ALTOS,NAO,ANOTA€OES EM BLOCOS E OU CADERNOS,NAO,NAO,NAO,NAO,NAO,NAO,FAMILIARES,NAO,193.60,1.0,20.0,100.0,2.0,16.0,8.5,750.0,15.0,2.0,2.0,18.0,1312.0,4.0,2.500000,21.250000,3.187500


In [15]:
select_df["NORM_FACTOR"] = (1000/select_df["CHEESE_PROXY"]).astype(int)

In [16]:
#Normalização

param_norm_div = ["S_21_AREA_PROPRIEDADE",
                "S_23_CASO_POSITIVO_QUANTOS_KVA",
                "S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO",
                "TOTAL_WORKFORCE",
                  "WHEY_DAY",
                  "CO_DAY",
                  "REVENUE_DAY",
                  "TAX_DAY"]

for i in param_norm_div:
    select_df[i+"{}".format("_NORM")] = round(select_df[i] / (1000/select_df["CHEESE_PROXY"]),2)

In [17]:
select_df.describe().T

,count,mean,std,min,25%,50%,75%,max
S_15_IDADE,757.0,52.376486,13.798613,18.000,42.000000,52.000000,62.00,92.000000
S_65_INFRAESTRUTURA_E_BENFEITORIAS_EXISTENTES,757.0,716.672391,415.645505,1.000,353.000000,717.000000,1073.00,1439.000000
S_21_AREA_PROPRIEDADE,757.0,91.557015,105.401733,3.000,24.200000,52.000000,110.00,556.000000
S_23_CASO_POSITIVO_QUANTOS_KVA,757.0,1.262880,0.440489,1.000,1.000000,1.000000,2.00,2.000000
S_50_PRODUÇÃO_TOTAL_DE_LEITE_DIA,757.0,140.433289,105.909989,2.000,60.000000,100.000000,200.00,450.000000
S_74_2_PORCENTAGEM_NA_FABRICAÇÃO_DO_QUEIJO,757.0,98.672391,12.415894,35.000,100.000000,100.000000,100.00,150.000000
S_78_2_PESO_TOTAL_DIA_QUEIJO_CANASTRA,757.0,16.831572,12.808792,1.000,7.000000,12.000000,22.00,60.000000
S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO,757.0,27.224571,19.231595,1.000,10.000000,25.000000,40.00,80.000000
S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA,757.0,8.754822,0.839938,7.000,8.000000,8.500000,9.00,13.000000
S_129_QUAL_RENDA_TOTAL_MENSAL_FAMILIAR,757.0,3794.791281,3618.733167,250.000,1000.000000,2500.000000,5000.00,20000.000000


## K Variables  Selection and Goals 

In [18]:
#K1 
#S_21_AREA_PROPRIEDADE_NORM MAX
#S_23_CASO_POSITIVO_QUANTOS_KVA_NORM MAX

#K2
#WHEY_DAY_NORM
#CO_DAY_NORM

#K3
#S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM MAX
#REVENUE_DAY_NORM MAX

#K4
#TOTAL_WORKFORCE_NORM MAX
#S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO MAX

#K5
#S_15_IDADE MIN
#TAX_DAY_NORM MAX

## General Constrains settings

In [19]:
IND_INIT_SIZE = 5

#K1

MAX_AREA = select_df["S_21_AREA_PROPRIEDADE_NORM"].quantile(0.95)
MAX_ELETR = select_df["S_23_CASO_POSITIVO_QUANTOS_KVA_NORM"].quantile(0.95)

MIN_AREA = select_df["S_21_AREA_PROPRIEDADE_NORM"].quantile(0.05)
MIN_ELETR = select_df["S_23_CASO_POSITIVO_QUANTOS_KVA_NORM"].quantile(0.05)

AREA_25 = select_df["S_21_AREA_PROPRIEDADE_NORM"].quantile(0.25)
ELETR_25 = select_df["S_23_CASO_POSITIVO_QUANTOS_KVA_NORM"].quantile(0.25)

AREA_75 = select_df["S_21_AREA_PROPRIEDADE_NORM"].quantile(0.75)
ELETR_75 = select_df["S_23_CASO_POSITIVO_QUANTOS_KVA_NORM"].quantile(0.75)


#K2

MAX_WHEY = select_df["WHEY_DAY_NORM"].quantile(0.95)
MAX_CO = select_df["CO_DAY_NORM"].quantile(0.95)

MIN_WHEY = select_df["WHEY_DAY_NORM"].quantile(0.05)
MIN_CO = select_df["CO_DAY_NORM"].quantile(0.05)

WHEY_25 = select_df["WHEY_DAY_NORM"].quantile(0.25)
CO_25 = select_df["CO_DAY_NORM"].quantile(0.25)

WHEY_75 = select_df["WHEY_DAY_NORM"].quantile(0.75)
CO_75 = select_df["CO_DAY_NORM"].quantile(0.75)


#K3

MAX_COW = select_df["S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM"].quantile(0.95)
MAX_REVENUE = select_df["REVENUE_DAY_NORM"].quantile(0.95)

MIN_COW = select_df["S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM"].quantile(0.05)
MIN_REVENUE = select_df["REVENUE_DAY_NORM"].quantile(0.05)

COW_25 = select_df["S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM"].quantile(0.25)
REVENUE_25 = select_df["REVENUE_DAY_NORM"].quantile(0.25)

COW_75 = select_df["S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM"].quantile(0.75)
REVENUE_75 = select_df["REVENUE_DAY_NORM"].quantile(0.75)

#K4

MAX_FORCE = select_df["TOTAL_WORKFORCE_NORM"].quantile(0.95)
MAX_TRADITION = select_df["S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO"].quantile(0.95)

MIN_FORCE = select_df["TOTAL_WORKFORCE_NORM"].quantile(0.05)
MIN_TRADITION = select_df["S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO"].quantile(0.05)

FORCE_25 = select_df["TOTAL_WORKFORCE_NORM"].quantile(0.25)
TRADITION_25 = select_df["S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO"].quantile(0.25)

FORCE_75 = select_df["TOTAL_WORKFORCE_NORM"].quantile(0.75)
TRADITION_75 = select_df["S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO"].quantile(0.75)

#K5

MAX_IDADE = select_df["S_15_IDADE"].quantile(0.95)
MAX_TAX = select_df["TAX_DAY_NORM"].quantile(0.95)

MIN_IDADE = select_df["S_15_IDADE"].quantile(0.05)
MIN_TAX = select_df["TAX_DAY_NORM"].quantile(0.05)

IDADE_25 = select_df["S_15_IDADE"].quantile(0.25)
TAX_25 = select_df["TAX_DAY_NORM"].quantile(0.25)

IDADE_75 = select_df["S_15_IDADE"].quantile(0.75)
TAX_75 = select_df["TAX_DAY_NORM"].quantile(0.75)

In [20]:
select_df[["S_21_AREA_PROPRIEDADE_NORM",
           "S_23_CASO_POSITIVO_QUANTOS_KVA_NORM",
           "WHEY_DAY_NORM",
           "CO_DAY_NORM",
           "S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM",
          "REVENUE_DAY_NORM",
           "TOTAL_WORKFORCE_NORM",
           "S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO",
           "S_15_IDADE",
           "TAX_DAY_NORM"
          ]].columns

Index(['S_21_AREA_PROPRIEDADE_NORM', 'S_23_CASO_POSITIVO_QUANTOS_KVA_NORM',
       'WHEY_DAY_NORM', 'CO_DAY_NORM',
       'S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM', 'REVENUE_DAY_NORM',
       'TOTAL_WORKFORCE_NORM', 'S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO',
       'S_15_IDADE', 'TAX_DAY_NORM'],
      dtype='object')

## Data input for Model

In [21]:
    #A ,KV ,WHEY  ,CO ,REB, REV,WOR,TEMP,AGE,TAX    

r1 = np.array([
    [600,4,20000,30000,22,13000,3,40,70,1750],  #same property, small tech, old one, 
    [500,5,16000,25000,15,7000,4,35,1000],     #same property, small tech, young one, 
    [400,7,15000,500000,15,8000,5,45,70,1200], # same property, medium tech,old one
    [500,7,14000,500000,14,8000,5,5,35,1100],  #same property, medium tech,young one
    [400,9,25000,25000,24,10000,11,3,29,1500],  #same property, high tech,young one
    [600,10,27000,25000,28,12000,10,25,60,2000]] #same property, high tech,old one
)


r = select_df[["S_21_AREA_PROPRIEDADE_NORM",
           "S_23_CASO_POSITIVO_QUANTOS_KVA_NORM",
           "WHEY_DAY_NORM",
           "CO_DAY_NORM",
           "S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM",
          "REVENUE_DAY_NORM",
           "TOTAL_WORKFORCE_NORM",
           "S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO",
           "S_15_IDADE",
           "TAX_DAY_NORM"
          ]].values

## Deap parameters inputting 

In [22]:
NBR_ITEMS = r.shape[0]

items = {}
# Create random items and store them in the items' dictionary.

#K1 
#S_21_AREA_PROPRIEDADE_NORM MAX
#S_23_CASO_POSITIVO_QUANTOS_KVA_NORM MAX

#K2
#WHEY_DAY_NORM MIN
#CO_DAY_NORM MIN

#K3
#S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM MAX
#S_129_QUAL_RENDA_TOTAL_MENSAL_FAMILIAR_NORM MAX

#K4
#TOTAL_WORKFORCE_NORM MAX
#S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO MAX

#K5
#S_15_IDADE MIN
#TAX_DAY_NORM MAX


for i in range(NBR_ITEMS):
    items[i] = (r[i][0] , r[i][1] , r[i][2] , r[i][3] , r[i][4] , r[i][5] , r[i][6], r[i][7], r[i][8],r[i][9])

creator.create("Fitness", base.Fitness, weights=(1,1,-1,-1,1,1,1,1,-1,1))  # Note here <- I used only two weights!  (at first, I tried weights=(-1.0 , -1.0, 1.0)) but it crashes. With deap, you cannot do such a thing.

creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, n=IND_INIT_SIZE) #
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

##  Data normalization objective function

In [23]:
def evaluation(individual):
    
    area = 0
    kva = 0
    whey = 0
    co = 0
    rebanho = 0
    renda = 0
    pessoas = 0 
    tradicao = 0
    idade = 0
    taxa = 0
    

    # Maximize or Minimize Conditions
    for item in individual:
        area += items[item][0]  # It must be maximized.
        kva += items[item][1]  # It must be maximized.
        whey += items[item][1]  # It must be minimized.
        co += items[item][1]  # It must be minimized.
        rebanho += items[item][2]  # It must be maximized.
        renda += items[item][3]  # It must be maximized.
        pessoas += items[item][4]  # It must be maximized.
        tradicao += items[item][5]  # It must be maximized.
        pessoas += items[item][6]  # It must be maximized.
        idade += items[item][7]  # It must be minimized.
        taxa += items[item][8]  # It must be maximized.


#MAXIMIZATION CONSTRAIN        
    
    if area > MAX_AREA:
        area = AREA_75
    if kva > MAX_ELETR:
        kva = ELETR_75
    if whey > MAX_WHEY:
        whey = WHEY_75
    if co > MAX_CO:
        co = CO_75
    if rebanho > MAX_COW:
        rebanho = COW_75
    if renda > MAX_REVENUE:
        renda = REVENUE_75
    if pessoas > MAX_FORCE:
        pessoas = FORCE_75
    if tradicao > MAX_TRADITION:
        tradicao = TRADITION_75
    if idade > MAX_IDADE:
        idade = IDADE_75
    if taxa > MAX_TAX:
        taxa = TAX_75

#MINIMIZATION CONSTRAIN
    
    if area < MIN_AREA:
        area = AREA_25
    if kva < MIN_ELETR:
        kva = ELETR_25
    if whey < MIN_WHEY:
        whey = WHEY_25
    if co < MIN_CO:
        co = CO_25
    if rebanho < MIN_COW:
        rebanho = COW_25
    if renda < MIN_REVENUE:
        renda = REVENUE_25
    if pessoas < MIN_FORCE:
        pessoas = FORCE_25
    if tradicao < MIN_TRADITION:
        tradicao = TRADITION_25
    if idade < MIN_IDADE:
        idade = IDADE_25
    if taxa < MIN_TAX:
        taxa = TAX_25

    MinFitess_score = idade + whey + co  # NOTE : Minimize weight, size
    MaxFitenss_score = area + kva + rebanho + renda + pessoas + tradicao + taxa # NOTE : Maximize weight, size
    
    return MinFitess_score , MaxFitenss_score

## Consolidation for Deap function 

In [24]:
def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)  # Used in order to keep type
    ind1 &= ind2  # Intersection (inplace)
    ind2 ^= temp  # Symmetric Difference (inplace)
    return ind1, ind2

def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:  # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,  # NOTE comma(,) , if there's no comma, an error occurs.

toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2) # NSGA-2 applies to multi-objective problems such as knapsack problem
toolbox.register("evaluate", evaluation)
    
def main():
    ngen = 1000 # a number of generation  < adjustable value >
    pop = toolbox.population(n= 1)
    hof = tools.ParetoFront() # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution
    
    stats = tools.Statistics(lambda ind: ind.fitness.values) #Stats fit
    
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0) 
    
    record = stats.compile(pop)
   
    pop, logbook = algorithms.eaSimple(pop, toolbox, 0.7, 0.2, ngen=ngen, stats=stats, halloffame=hof, verbose=False)
    
    return pop,logbook,hof

if __name__ == "__main__":
    pop,logbook,hof = main()
    
    print(hof) # non-dominated individuals' list  # the fittest value is placed on the most right side.

[Individual({133, 264, 265, 654, 275, 33, 302, 688, 561, 695, 187, 317, 575, 320, 582, 589, 590, 87, 735, 356, 753, 242}), Individual({33, 642, 195, 38, 264, 561, 18, 275, 500, 563, 277, 695, 635, 412, 311, 735}), Individual({195, 38, 231, 41, 561, 18, 275, 274, 468, 341, 695, 213, 466, 412, 311, 735}), Individual({195, 231, 8, 41, 695, 399, 561, 274, 275, 468, 466, 54, 279, 472, 597, 189, 412, 311}), Individual({292, 613, 427, 205, 272, 689})]


In [34]:
#loop for brutal force

lista_record=[]

for i in tqdm(range(0,1000)):

    ngen = i  # a number of generation  < adjustable value >

    pop = toolbox.population(n=1)
    hof = tools.ParetoFront() # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values) #Stats fit
    stats_size = tools.Statistics(key=len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)

    mstats.register("avg", np.mean, axis=0)
    mstats.register("std", np.std, axis=0)
    mstats.register("min", np.min, axis=0)
    mstats.register("max", np.max, axis=0) 

    pop, logbook = algorithms.eaSimple(pop, toolbox, 0.7, 0.2, ngen=ngen, stats=mstats, halloffame=hof, verbose=False)
    
    #pop, logbook = algorithms.eaMuPlusLambda(pop, toolbox, mu = 20,lambda_=10 ,cxpb=0.7, mutpb=0.2, ngen=ngen, stats=mstats, halloffame=hof, verbose=False)
    
    #pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu = 20,lambda_=10 ,cxpb=0.7, mutpb=0.2, ngen=ngen, stats=mstats, halloffame=hof, verbose=False)
    
    #pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu = 20,lambda_=10 ,cxpb=0.7, mutpb=0.2, ngen=ngen, stats=mstats, halloffame=hof, verbose=False)

    record = mstats.compile(pop)
    lista_record.append(record)


 35%|███▍      | 346/1000 [00:22<01:47,  6.06it/s]


 55%|█████▌    | 551/1000 [00:44<00:54,  8.30it/s]


 70%|███████   | 703/1000 [01:02<00:41,  7.23it/s]


 85%|████████▌ | 854/1000 [01:24<00:23,  6.32it/s]


100%|██████████| 1000/1000 [01:50<00:00,  5.21it/s]

In [27]:
lista=[]
for j in range(0,len(hof)):
    for i in hof[j]:
        lista.append(i)

In [31]:
#13 para uma unidade geracional fruto com 100 de loop
#23 para uma unidade geracional fruto com 1000 de loop
#77 para uma unidade geracional fruto com 1000 de loop

In [29]:
select_df.loc[select_df.index[lista]].shape

(19, 51)

In [30]:
select_df.loc[select_df.index[lista]].drop_duplicates("S_13_NOME_PROPRIETARIO").shape

(13, 51)

In [32]:
#SELECTED HOF

input_1=select_df.loc[select_df.index[lista]][["S_21_AREA_PROPRIEDADE_NORM",
           "S_23_CASO_POSITIVO_QUANTOS_KVA_NORM",
           "WHEY_DAY_NORM",
           "CO_DAY_NORM",
           "S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM",
          "REVENUE_DAY_NORM",
           "TOTAL_WORKFORCE_NORM",
           "S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO",
           "S_15_IDADE",
           "TAX_DAY_NORM"]].mean().values  



#NOT SELECTED HOF

input_2=select_df.loc[select_df.index.difference(lista)][["S_21_AREA_PROPRIEDADE_NORM",
           "S_23_CASO_POSITIVO_QUANTOS_KVA_NORM",
           "WHEY_DAY_NORM",
           "CO_DAY_NORM",
           "S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM",
          "REVENUE_DAY_NORM",
           "TOTAL_WORKFORCE_NORM",
           "S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO",
           "S_15_IDADE",
           "TAX_DAY_NORM"]].mean().values  

In [33]:
#Dataframe de comparação 

columns = ["S_21_AREA_PROPRIEDADE_NORM",
           "S_23_CASO_POSITIVO_QUANTOS_KVA_NORM",
           "WHEY_DAY_NORM",
           "CO_DAY_NORM",
           "S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM",
          "REVENUE_DAY_NORM",
           "TOTAL_WORKFORCE_NORM",
           "S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO",
           "S_15_IDADE",
           "TAX_DAY_NORM"]

pd.DataFrame([input_1,input_2],columns= columns,).T.rename(columns={0:"Selected",1:"Not_Selected"})


,Selected,Not_Selected
S_21_AREA_PROPRIEDADE_NORM,2.291579,1.657258
S_23_CASO_POSITIVO_QUANTOS_KVA_NORM,0.034211,0.023105
WHEY_DAY_NORM,6.670526,3.574180
CO_DAY_NORM,62.201579,42.071707
S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO_NORM,0.782105,0.529032
REVENUE_DAY_NORM,8.410526,4.643844
TOTAL_WORKFORCE_NORM,0.086316,0.067460
S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO,15.210526,17.369624
S_15_IDADE,49.947368,52.337366
TAX_DAY_NORM,1.382105,0.796156


In [ ]:
gen = logbook.select("gen")

fit_mins = [lista_record[n]["fitness"]["min"][0] for n in range(0,len(lista_record))]

size_avgs = [lista_record[n]["size"]["avg"] for n in range(0,len(lista_record))]

fit_maxs = [lista_record[n]["fitness"]["max"][0] for n in range(0,len(lista_record))]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


fig, ax1 = plt.subplots()
line1 = ax1.plot(gen, fit_mins, "b-", label="Minimum Fitness")
ax1.set_xlabel("Generation")
ax1.set_ylabel("Fitness", color="b")
for tl in ax1.get_yticklabels():
    tl.set_color("b")

ax2 = ax1.twinx()
line2 = ax2.plot(gen, size_avgs, "r-", label="Average Size")
ax2.set_ylabel("Size", color="r")
for tl in ax2.get_yticklabels():
    tl.set_color("r")

lns = line1 + line2
labs = [l.get_label() for l in lns]

ax2.legend(lns, labs, loc="center right", bbox_to_anchor=(1.5, 0.5))

plt.show()

In [ ]:
def EvaluateObjective(individual):
    prediction = calculate(individual, for_sensitivity)
    prediction = [int(i) for i in prediction]

    # diff = []
    # for y in range(len(targets)):
    #     output = math.sqrt((targets[y] - prediction[y]) ** 2)
    #     #output = abs(targets[y] - prediction[y])
    #     diff.append(output)

    rmse = np.sqrt((sum((i - j)**2 for i, j in zip(prediction, targets)) / len(targets)))

    return (rmse,)

## Further Analysis for Thesis

Primeiro estudo Ranking usando poucos produtores com diferentes graus tecnológicos no goal programming - do Biagio

Segundo estudo Classificação dos clusters com outras variáveis e e o goal programming - do Biagio

Terceiro estudo - Base inteira e tentar encontrar uma curva de pareto com produtores da fronteira via algoritmo genético

Quarto estudo - 



